## Sequence Runnable

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser

In [7]:
load_dotenv()

prompt_1 = PromptTemplate(
    template="Tell me a good joke on the topic of {topic}.", 
    input_variables=["topic"],
)

prompt_2 = PromptTemplate(
    template="Explain the joke: {joke}",
    input_variables=["joke"],
)

llm = ChatOpenAI()

parser = StrOutputParser()

runnable = RunnableSequence(prompt_1, llm, parser, prompt_2, llm, parser)

output = runnable.invoke({"topic": "AI"})

print(output)

This joke is a play on words. The term "hard drive" is typically used to refer to the storage device in computers that store data. In this joke, it is being used in a literal sense to imply that the robot had a hard time driving (or hitting) the tennis ball, hence why it was bad at tennis.


## Parallel Runnable

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

prompt_1 = PromptTemplate(
    template = "Generate a tweet on this topic: {topic}",
    input_variables = ["topic"],    
)

prompt_2 = PromptTemplate(
    template = "Generate a LinkedIn post on this topic: {topic}",
    input_variables = ["topic"],    
)

llm = ChatOpenAI()

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    "tweet": RunnableSequence(prompt_1, llm, parser),
    "linkedin_post": RunnableSequence(prompt_2, llm, parser),
})

output = parallel_chain.invoke({'topic': 'Programming'})

print(output)

{'tweet': '"Just finished debugging my code and feeling like a coding wizard 💻✨ #programming #codinglife"', 'linkedin_post': "Excited to share my latest project in programming - I just finished creating a new web application from scratch using HTML, CSS, and JavaScript. It was challenging but incredibly rewarding to see it come to life. Can't wait to continue learning and growing in the programming world. #programming #webdevelopment #codingjourney"}


## Passthrough Runnable

In [10]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableSequence
from langchain_core.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI()

prompt_1 = PromptTemplate(
    template="Generate a joke for me in this topic: {topic}",
    input_variables=["topic"]
)

parser = StrOutputParser()

prompt_2 = PromptTemplate(
    template="Explain this joke to me: {joke}",
    input_variables=["joke"]
)

joke_gen_chain = RunnableSequence(prompt_1, llm, parser)

parallel_chain = RunnableParallel({
    'Joke': RunnablePassthrough(),
    "explain": RunnableSequence(prompt_2, llm, parser)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

output = final_chain.invoke({'topic': 'AI'})

print(output)

{'Joke': 'Why did the robot go to therapy?\nBecause it had too many bytes of emotional baggage!', 'explain': 'This joke is a play on words combining technology terms and emotional concepts. In the joke, "bytes" is a unit of measurement for data in computing, but it is used humorously here to refer to emotional baggage. The idea is that the robot had accumulated too much emotional baggage, so it needed to go to therapy to unpack and deal with it.'}


## Lambda Runnable
* To convert python function into runnable.
- eg: Make a function in python which clean the data. And you can convert that output into runnable. 

In [11]:
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

def number_words(sentence: str):
    return len(sentence.split())

llm = ChatOpenAI()

prompt_1 = PromptTemplate(
    template = "Write a joke on this topic: {topic}",
    input_variables= ["topic"]
)

parser = StrOutputParser()

gen_joke_chain = RunnableSequence(prompt_1, llm, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(number_words)
    #'word_count': RunnableLambda(lambda x: len(x.split()))
})

final_chain = RunnableSequence(gen_joke_chain, parallel_chain)

output = final_chain.invoke({'topic': 'AI'})

print(output)

{'joke': "Why did the AI break up with the calculator?\n\nBecause it couldn't handle its division problems!", 'word_count': 16}


## Branch Runnable

* for conditional chain: (if, else)

In [13]:
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt_1 = PromptTemplate(
    template= "Generate a detail report on this topic: {topic}",
    input_variables=["topic"]
)

prompt_1_1 = PromptTemplate(
    template= "Give me the summary of this report: {report}",
    input_variables=['report']
)

llm = ChatOpenAI()

parser = StrOutputParser()

detail_report = RunnableSequence(prompt_1, llm, parser)

branch_chain = RunnableBranch(
    (lambda x: len(x.split()) > 500, RunnableSequence(prompt_1_1, llm, parser)),
    RunnablePassthrough()
)

final_chain = RunnableSequence(detail_report, branch_chain)

output = final_chain.invoke({'topic': 'India vs australia cricket match'})

print(output)

India vs Australia cricket match

Date: November 27, 2021
Venue: Sydney Cricket Ground, Sydney, Australia

Introduction:
The highly anticipated cricket match between India and Australia took place on November 27, 2021 at the Sydney Cricket Ground in Australia. The match drew in a large audience of cricket fans from both countries, eager to witness the intense rivalry between two of the biggest cricketing nations in the world.

First innings:
Australia won the toss and elected to bat first. Their top order batsmen got off to a solid start, with openers David Warner and Aaron Finch setting the foundation for a big total. Warner played a stellar innings, scoring a century and anchoring the Australian innings. However, India's bowlers made a strong comeback in the later overs, picking up crucial wickets and restricting Australia to a total of 285/6 in their allotted 50 overs.

Second innings:
In response, India got off to a shaky start, losing a couple of quick wickets early on. However, c

## LCEL: langchain expression language.

- RunnableSequence(prompt, llm, parser) --> prompt | llm | parser

In [14]:
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

def number_words(sentence: str):
    return len(sentence.split())

llm = ChatOpenAI()

prompt_1 = PromptTemplate(
    template = "Write a joke on this topic: {topic}",
    input_variables= ["topic"]
)

parser = StrOutputParser()

gen_joke_chain = prompt_1 | llm | parser            # RunnableSequence(prompt_1, llm, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(number_words)
    #'word_count': RunnableLambda(lambda x: len(x.split()))
})

final_chain =  gen_joke_chain | parallel_chain         # RunnableSequence(gen_joke_chain, parallel_chain)

output = final_chain.invoke({'topic': 'AI'})

print(output)

{'joke': "Why did the artificial intelligence break up with its computer? Because it couldn't handle their motherboard's emotional baggage!", 'word_count': 18}
